# The fertility model in a Malthus Economy

> **Note the following:** 
> 1. This is *not* meant to be an example of an actual **model analysis project**, just an example of how to structure such a project.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `modelproject.py` file includes a function which could be used multiple times in this notebook.

Imports and set magics:

In [1]:
import numpy as np
from scipy import optimize
import sympy as sm

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
import modelproject

# Model description

We consider a Malthus economy where we apply fertility as the main factor of this model. 

The utility function of household depends on consumption $c_t$ and number of surviving children $n_t$ where $\beta \epsilon (0.1)$.  

$$u_t(c_t) = \beta\ln(c_t) + (1-\beta)\ln(n_t)$$

The budget constraint is given as:

$$c_t + \lambda n_t = y_t (1-\tau)$$

Where c is consumption, $\lambda$ is relative price on 1 child, $y_t$ is the average income and $\tau$ is income after tax. 

Income per capita is given by: 
$y_t \equiv (\frac{AX}{L_t})^{\alpha}$

We solve the problem by inserting the budget contraint in the utility function: 

$$\max_{u_t}=\beta log[y_t (1-\tau)-\lambda n_t]+(1-\beta) log (n_t)$$

We take FOC wrt. $n_t$
$$\frac{\partial u}{\partial n}=0: \beta \frac{-\lambda}{y_t (1-\tau)-\lambda n_t}+(1-\beta) \frac{1}{n_t}=0$$

Isolating $n_t$ yields:
$$n_t=\frac{1-\beta}{\lambda}y_t(1-\tau)$$

Where:
$$n_t=\frac{1-\beta}{\lambda}y_t(1-\tau) \equiv \eta y_t$$

And $$\eta \equiv\frac{1-\beta}{\lambda}(1-\tau)$$

Population in the next period is the number of surviving children, and the the part of the population that survives to the next period: <br>
$$L_{t+1}=n_tL_t+(1-\mu)L_t$$
Where $\mu$ is adult mortality. 

## Analytical solution

If your model allows for an analytical solution, you should provide here.

You may use Sympy for this. Then you can characterize the solution as a function of a parameter of the model.

To characterize the solution, first derive a steady state equation as a function of a parameter using Sympy.solve and then turn it into a python function by Sympy.lambdify. See the lecture notes for details. 

## Numerical solution

You can always solve a model numerically. 

Define first the set of parameters you need. 

Then choose one of the optimization algorithms that we have gone through in the lectures based on what you think is most fitting for your model.

Are there any problems with convergence? Does the model converge for all starting values? Make a lot of testing to figure these things out. 

# Further analysis

Make detailed vizualizations of how your model changes with parameter values. 

Try to make an extension of the model. 

# Conclusion

Add concise conclusion. 